# Q2.1 Classic Machine Learning Methods

In [1]:
import pandas as pd
import numpy as np
from project_1.config import PROCESSED_DATA_DIR, PROJ_ROOT

SEED = 42

2025-03-26 15:39:58.582 | INFO     | project_1.config:<module>:11 - PROJ_ROOT path is: /Users/francescobondi/Desktop/stuff/ETH/FS25/ML for Healthcare/project-1-ml4hc


In [2]:
# Load the data from Parquet files
sets_dict = {}
sets = ["a", "b", "c"]

for set_name in sets:
    directory = PROCESSED_DATA_DIR / f"set_{set_name}_final.parquet"
    temp_set = pd.read_parquet(directory)
    sets_dict[f"set_{set_name}"] = temp_set

# Assure the loading was correct
print(sets_dict["set_a"].shape)
sets_dict["set_a"].head(10)


(196000, 43)


,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,Urine,WBC,pH,MechVent,TroponinT,ALP,ALT,AST,Bilirubin,TroponinI
0,132539.0,2025-03-10 00:00:00,0.0,-0.950365,-0.231946,-0.583612,1.452191,-0.133234,-0.810049,-0.037519,...,0.923077,0.318841,-0.250,0.0,1.314286,0.189655,-0.307692,0.037037,1.166667,0.857143
1,132539.0,2025-03-10 01:00:00,0.0,-0.950365,-0.231946,-0.583612,0.171208,-0.693421,0.634509,-2.669841,...,3.076923,0.608696,-0.625,0.0,14.571429,-0.500000,1.557692,-0.240741,0.416667,-0.214286
2,132539.0,2025-03-10 02:00:00,0.0,-0.950365,-0.231946,-0.583612,1.025197,0.856429,0.937401,-0.256879,...,-0.384615,0.434783,0.000,0.0,0.542857,2.120690,-0.461538,1.777778,0.083333,-0.178571
3,132539.0,2025-03-10 03:00:00,0.0,-0.950365,-0.231946,-0.583612,-0.398117,2.761064,0.587911,-0.037519,...,0.692308,-0.594203,0.625,0.0,2.942857,0.534483,51.769231,-0.259259,22.750000,0.357143
4,132539.0,2025-03-10 04:00:00,0.0,-0.950365,-0.231946,-0.583612,-0.540449,-0.002524,0.937401,0.181842,...,-0.153846,-0.028986,0.125,0.0,0.857143,-0.224138,-0.096154,-0.018519,1.416667,0.785714
5,132539.0,2025-03-10 05:00:00,0.0,-0.950365,-0.231946,-0.583612,-0.113454,-1.365645,-0.320763,-0.914959,...,-0.153846,-0.637681,-0.875,0.0,-0.028571,2.189655,-0.403846,-0.240741,-0.583333,0.071429
6,132539.0,2025-03-10 06:00:00,0.0,-0.950365,-0.231946,-0.583612,1.167528,-0.133234,-0.285814,-1.792400,...,0.692308,0.884058,0.250,0.0,-0.085714,-0.189655,-0.057692,-0.314815,0.750000,1.857143
7,132539.0,2025-03-10 07:00:00,0.0,-0.950365,-0.231946,-0.583612,-0.682780,1.453962,0.937401,-0.914959,...,0.692308,-0.623188,0.500,0.0,-0.228571,-0.500000,-0.173077,0.074074,-0.166667,1.035714
8,132539.0,2025-03-10 08:00:00,0.0,-0.950365,-0.231946,-0.583612,1.167528,-0.898823,-0.600355,0.401202,...,0.307692,-0.173913,-1.250,0.0,0.371429,-0.482759,3.538462,-0.203704,-0.166667,0.392857
9,132539.0,2025-03-10 09:00:00,0.0,-0.950365,-0.231946,-0.583612,0.171208,-0.394655,1.776177,1.278643,...,0.000000,-1.333333,0.500,0.0,10.285714,0.068966,-0.461538,-0.111111,30.666667,-0.250000


ICU TYpe da cavare

In [3]:

# Define file names
file_names = ["Outcomes-a.txt", "Outcomes-b.txt", "Outcomes-c.txt"]

# Directory path
base_path = PROJ_ROOT / "data" / "data_1" / "predicting-mortality-of-icu-patients-the-physionet-computing-in-cardiology-challenge-2012-1.0.0"

# Read files into DataFrames containing all variables
outcomes_a, outcomes_b, outcomes_c = [pd.read_csv(base_path / name) for name in file_names]

# Extract only the "RecordID" and "In-hospital_death" column into separate DataFrames
death_a, death_b, death_c = [df[["RecordID", "In-hospital_death"]] for df in [outcomes_a, outcomes_b, outcomes_c]]
death_a.head()

#CHECK for missing values in the outcome data
print(death_a.isnull().sum())
print(death_b.isnull().sum())
print(death_c.isnull().sum())



RecordID             0
In-hospital_death    0
dtype: int64
RecordID             0
In-hospital_death    0
dtype: int64
RecordID             0
In-hospital_death    0
dtype: int64


In [4]:
# Check if each "In-hospital_death" column contains only 0 and 1
for name, df in zip(["a", "b", "c"], [death_a, death_b, death_c]):
    print(f"\nValue counts for death_{name}:\n")
    print(df["In-hospital_death"].value_counts())
    print("\nContains only 0 and 1:", df["In-hospital_death"].isin([0, 1]).all())



Value counts for death_a:

In-hospital_death
0    3446
1     554
Name: count, dtype: int64

Contains only 0 and 1: True

Value counts for death_b:

In-hospital_death
0    3432
1     568
Name: count, dtype: int64

Contains only 0 and 1: True

Value counts for death_c:

In-hospital_death
0    3415
1     585
Name: count, dtype: int64

Contains only 0 and 1: True


So 1 means he died. We are going to fit a classifier to predict death. 

### Initial setup for the classifier to work

In [5]:
# Define the sets you want to modify
sets = ["set_a", "set_b", "set_c"]

# Delete the column ICUType from each set
for set_name in sets:
    sets_dict[set_name].drop(columns=["ICUType"], inplace=True)

# Check if the column was deleted by printing all the columns of set_a
sets_dict["set_a"].columns

Index(['RecordID', 'Time', 'Gender', 'Height', 'Weight', 'Age', 'Albumin',
       'Cholesterol', 'DiasABP', 'HCO3', 'HCT', 'HR', 'Mg', 'MAP', 'Na',
       'NIDiasABP', 'NIMAP', 'NISysABP', 'SysABP', 'PaCO2', 'PaO2',
       'Platelets', 'RespRate', 'Temp', 'BUN', 'Creatinine', 'FiO2', 'GCS',
       'Glucose', 'K', 'Lactate', 'SaO2', 'Urine', 'WBC', 'pH', 'MechVent',
       'TroponinT', 'ALP', 'ALT', 'AST', 'Bilirubin', 'TroponinI'],
      dtype='object')

In [6]:
# Define training, validation and testing sets
train_set = sets_dict["set_a"]
val_set = sets_dict["set_b"]
test_set = sets_dict["set_c"]


In [7]:

#change class of RecordID to int32
train_set["RecordID"] = train_set["RecordID"].astype("int32")
val_set["RecordID"] = val_set["RecordID"].astype("int32")
test_set["RecordID"] = test_set["RecordID"].astype("int32")


In [8]:
# Merge the training, validation and testing sets with the corresponding death DataFrames
train_set = train_set.merge(death_a, on="RecordID")
val_set = val_set.merge(death_b, on="RecordID") 
test_set = test_set.merge(death_c, on="RecordID")

# Check if the merge was successful by printing the first 5 rows of the training set
train_set.head()

,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,...,WBC,pH,MechVent,TroponinT,ALP,ALT,AST,Bilirubin,TroponinI,In-hospital_death
0,132539,2025-03-10 00:00:00,0.0,-0.950365,-0.231946,-0.583612,1.452191,-0.133234,-0.810049,-0.037519,...,0.318841,-0.250,0.0,1.314286,0.189655,-0.307692,0.037037,1.166667,0.857143,0
1,132539,2025-03-10 01:00:00,0.0,-0.950365,-0.231946,-0.583612,0.171208,-0.693421,0.634509,-2.669841,...,0.608696,-0.625,0.0,14.571429,-0.500000,1.557692,-0.240741,0.416667,-0.214286,0
2,132539,2025-03-10 02:00:00,0.0,-0.950365,-0.231946,-0.583612,1.025197,0.856429,0.937401,-0.256879,...,0.434783,0.000,0.0,0.542857,2.120690,-0.461538,1.777778,0.083333,-0.178571,0
3,132539,2025-03-10 03:00:00,0.0,-0.950365,-0.231946,-0.583612,-0.398117,2.761064,0.587911,-0.037519,...,-0.594203,0.625,0.0,2.942857,0.534483,51.769231,-0.259259,22.750000,0.357143,0
4,132539,2025-03-10 04:00:00,0.0,-0.950365,-0.231946,-0.583612,-0.540449,-0.002524,0.937401,0.181842,...,-0.028986,0.125,0.0,0.857143,-0.224138,-0.096154,-0.018519,1.416667,0.785714,0


In [9]:
# Define aggregation rules
aggregation_rules = {
    "Age": "last",
    "Gender": "last",
    "Height": "last",
    "Albumin": "last",
    "ALP": "last",
    "ALT": "last",
    "AST": "last",
    "Bilirubin": "last",
    "BUN": "last",
    "Cholesterol": "last",
    "Creatinine": "last",
    "DiasABP": "mean",
    "FiO2": "mean",
    "GCS": "min",
    "Glucose": "mean",
    "HCO3": "last",
    "HCT": "last",
    "HR": "mean",
    "K": "last",
    "Lactate": "max",
    "Mg": "last",
    "MAP": "mean",
    "MechVent": "last",
    "Na": "last",
    "NIDiasABP": "mean",
    "NIMAP": "mean",
    "NISysABP": "mean",
    "PaCO2": "last",
    "PaO2": "mean",
    "pH": "last",
    "Platelets": "last",
    "RespRate": "mean",
    "SaO2": "mean",
    "SysABP": "mean",
    "Temp": "max",
    "TroponinI": "max",
    "TroponinT": "max",
    "Urine": "sum",
    "WBC": "last",
    "Weight": "last",
    "In-hospital_death": "max"  # If any 1 exists for a patient, return 1
}

# Perform aggregation
train_aggregated = train_set.groupby("RecordID").agg(aggregation_rules).reset_index()
val_aggregated = val_set.groupby("RecordID").agg(aggregation_rules).reset_index()
test_aggregated = test_set.groupby("RecordID").agg(aggregation_rules).reset_index()


# Display the processed dataset
train_aggregated.head()

,RecordID,Age,Gender,Height,Albumin,ALP,ALT,AST,Bilirubin,BUN,...,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,In-hospital_death
0,132539,-0.583612,0.0,-0.950365,-1.252106,-0.586207,-0.461538,1.259259,-0.250000,-0.588235,...,-0.322617,-0.979592,-0.152341,2.504504,9.714286,16.8,35.615385,-0.304348,-0.231946,0
1,132540,0.669324,1.0,0.618599,-1.252106,-0.586207,-0.461538,1.259259,-0.250000,0.176471,...,-0.003777,-1.285714,-0.008614,2.504504,9.714286,16.8,26.985897,0.260870,-0.234578,0
2,132541,-1.153129,0.0,-0.429179,-1.109774,0.500000,0.807692,2.462963,1.666667,-0.882353,...,-0.003777,-2.142857,-0.033771,2.504504,9.714286,16.8,21.969231,-0.768116,-1.081104,0
3,132543,0.213711,1.0,1.159248,1.879185,0.500000,-0.403846,-0.296296,-0.500000,-0.470588,...,-0.712056,-0.979592,-0.152341,2.504504,9.714286,16.8,135.250000,-0.521739,0.142631,0
4,132545,1.352744,0.0,-1.252047,0.313540,-0.586207,-0.461538,1.259259,-0.250000,0.411765,...,-0.048940,-0.979592,-0.152341,2.504504,9.714286,16.8,-4.300000,-0.971014,-0.812672,0


# Model 1 - Logistic Regression

In [10]:
# Separate Predictors (X) and Target (y)
X_train = train_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_train = train_aggregated["In-hospital_death"]

X_val = val_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_val = val_aggregated["In-hospital_death"]

X_test = test_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_test = test_aggregated["In-hospital_death"]

# Visualize the shape of the datasets
print(X_train.shape, y_train.shape)


(4000, 40) (4000,)


In [11]:
#El bondiano fa una logistica daje
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

# --- Assume these DataFrames are already defined ---
# train_df = pd.read_csv("train.csv")
# valid_df = pd.read_csv("validate.csv")
# test_df  = pd.read_csv("test.csv")

# Separate features and target for each set.
# Adjust the column name "target" to your actual target column name.

# Create and train the Logistic Regression classifier.
clf = LogisticRegression(random_state=SEED, max_iter=1000)
clf.fit(X_train, y_train)

# Optionally, evaluate on the validation set.
y_valid_proba = clf.predict_proba(X_val)[:, 1]  # probability for the positive class
roc_auc_valid = roc_auc_score(y_val, y_valid_proba)
auprc_valid = average_precision_score(y_val, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on the test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.786, AUPRC: 0.408
Test ROC AUC: 0.758, AUPRC: 0.368


# Model 2 - Random Forest

In [12]:
# Use Random Forest to predict the target
from sklearn.ensemble import RandomForestClassifier

# Create and train the Random Forest classifier.
clf = RandomForestClassifier(random_state=SEED)
clf.fit(X_train, y_train)

# Optionally, evaluate on the validation set.
y_valid_proba = clf.predict_proba(X_val)[:, 1]  # probability for the positive class
roc_auc_valid = roc_auc_score(y_val, y_valid_proba)
auprc_valid = average_precision_score(y_val, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on the test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.781, AUPRC: 0.421
Test ROC AUC: 0.739, AUPRC: 0.392


## A little comment on the results:

AUC Score is pretty good. However, AUPRC is not that high. A problem that could arise is class imbalance. In fact, we have two classes, where the size of the 0 class is 6x the size of the 1 class. However, all three sets share the same problem, so it should not be a big problem. Another possible implementation could be using a dimensionality reduction technique, or some feature engineering.

# Q2.1 - Feature Engineering

We could use solutions such as *_Feature Lagging_*, *_Temporal Differences_* or *_Rolling Statistics_* to better capture the time-series trends.

### Attempt 2 - Signal processing-based features

Use tsfresh to extract a large amount of features. 

*Note* the use of MinimalFCParameters, to reduce the number of variables inspected (and so the high computational cost) 

Also tryied to substitute w/ EfficientFCParameters but still the run time is too much.

Maybe we should try to use the student cluster but we'll see



In [13]:
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction.settings import MinimalFCParameters
from tsfresh.utilities.dataframe_functions import impute

# Define the column names
id_column = "RecordID"
time_column = "Time"
target_column = "In-hospital_death"

# Step 1: Extract all features from train_set using MinimalFCParameters
X_train_extracted = extract_features(
    train_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use minimal features
)

# Impute missing values (tsfresh requires no NaNs)
X_train_extracted = impute(X_train_extracted)

# Step 2: Select relevant features based on the target variable
y_train = train_set.groupby(id_column)[target_column].max()
X_train_relevant = select_features(X_train_extracted, y_train)

# Step 3: Store the relevant feature names
relevant_feature_names = X_train_relevant.columns

# Step 4: Extract the same features for val_set
X_val_extracted = extract_features(
    val_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use the same settings
)
X_val_extracted = impute(X_val_extracted)  # Impute missing values
X_val_relevant = X_val_extracted[relevant_feature_names]  # Keep only relevant features

# Step 5: Extract the same features for test_set
X_test_extracted = extract_features(
    test_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use the same settings
)
X_test_extracted = impute(X_test_extracted)  # Impute missing values
X_test_relevant = X_test_extracted[relevant_feature_names]  # Keep only relevant features

# Now X_train_relevant, X_val_relevant, and X_test_relevant have the same features
print("Train set relevant features:", X_train_relevant.shape)
print("Validation set relevant features:", X_val_relevant.shape)
print("Test set relevant features:", X_test_relevant.shape)

Feature Extraction: 100%|██████████| 20/20 [00:09<00:00,  2.21it/s]


Train set relevant features: (4000, 203)
Validation set relevant features: (4000, 203)
Test set relevant features: (4000, 203)


In [14]:
# For train_set
y_train = train_set.groupby(id_column)[target_column].max()

# For val_set
y_val = val_set.groupby(id_column)[target_column].max()

# For test_set
y_test = test_set.groupby(id_column)[target_column].max()


### Evaluation with Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler

# --- Scale the features ---
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train_scaled = scaler.fit_transform(X_train_relevant)

# Transform the validation and test sets using the same scaler
X_val_scaled = scaler.transform(X_val_relevant)
X_test_scaled = scaler.transform(X_test_relevant)

# --- Create and train the Logistic Regression classifier ---
clf = LogisticRegression(random_state=42, max_iter=5000)  # Increase iterations to 5000
clf.fit(X_train_scaled, y_train)

# --- Evaluate on the validation set ---
y_val_proba = clf.predict_proba(X_val_scaled)[:, 1]  # Probability for the positive class
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set ---
y_test_proba = clf.predict_proba(X_test_scaled)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")


Validation ROC AUC: 0.804, AUPRC: 0.410
Test ROC AUC: 0.770, AUPRC: 0.343


Since there is a total of 215 features exctracted, we'll try with some regularization.

### Evaluation with Logistic Regression (l2-reg)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# --- Scale the features ---
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train_scaled = scaler.fit_transform(X_train_relevant)

# Transform the validation and test sets using the same scaler
X_val_scaled = scaler.transform(X_val_relevant)
X_test_scaled = scaler.transform(X_test_relevant)

# --- Define the Logistic Regression model ---
# Use L2 regularization (default) and set up the solver
log_reg = LogisticRegression(random_state=42, max_iter=5000, penalty='l2', solver='liblinear')

# --- Set up the grid search for hyperparameter tuning ---
# Define the range of C values to test
param_grid = {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}

# Use ROC AUC as the scoring metric for grid search
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=1)

# --- Perform grid search on the training data ---
grid_search.fit(X_train_scaled, y_train)

# --- Get the best model and its parameters ---
best_clf = grid_search.best_estimator_
best_C = grid_search.best_params_['C']
print(f"Best regularization parameter (C): {best_C}")

# --- Evaluate on the validation set using the best model ---
y_val_proba = best_clf.predict_proba(X_val_scaled)[:, 1]  # Probability for the positive class
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set using the best model ---
y_test_proba = best_clf.predict_proba(X_test_scaled)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best regularization parameter (C): 0.01
Validation ROC AUC: 0.830, AUPRC: 0.450
Test ROC AUC: 0.803, AUPRC: 0.391


### Evaluation with Logistic Regression (l1- reg)

Since Lasso convergence is slower, we improved it by using the saga solver, increasing the iterations, adjusting the tolerance and also doing the grid search with every processor (n_jobs=-1)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# --- Scale the features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_relevant)
X_val_scaled = scaler.transform(X_val_relevant)
X_test_scaled = scaler.transform(X_test_relevant)

# --- Define the Logistic Regression model with L1 regularization ---
log_reg = LogisticRegression(
    penalty='l1', 
    solver='saga', 
    max_iter=5000,  # Increase max_iter
    tol=1e-3,       # Adjust tolerance
    warm_start=True, # Enable warm start
    random_state=42
)
# --- Set up the grid search for hyperparameter tuning ---
param_grid = {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)

# --- Perform grid search on the training data ---
grid_search.fit(X_train_scaled, y_train)

# --- Get the best model and its parameters ---
best_clf = grid_search.best_estimator_
best_C = grid_search.best_params_['C']
print(f"Best regularization parameter (C): {best_C}")

# --- Evaluate on the validation set using the best model ---
y_val_proba = best_clf.predict_proba(X_val_scaled)[:, 1]
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set using the best model ---
y_test_proba = best_clf.predict_proba(X_test_scaled)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best regularization parameter (C): 0.1
Validation ROC AUC: 0.832, AUPRC: 0.457
Test ROC AUC: 0.811, AUPRC: 0.403


### Evaluation with Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np

# --- Define the Random Forest Classifier ---
rf = RandomForestClassifier(random_state=42, n_jobs=-1)  # Use all CPU cores

# --- Set up the randomized search for hyperparameter tuning ---
param_dist = {
    'n_estimators': [100, 200],  # Number of trees
    'max_depth': [10, 20],       # Maximum depth of each tree
    'min_samples_split': [2, 5], # Minimum samples to split a node
    'min_samples_leaf': [1, 2],  # Minimum samples at each leaf node
    'max_features': ['sqrt', 'log2']  # Number of features to consider at each split
}

# Use RandomizedSearchCV with a fixed number of iterations
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,  # Number of hyperparameter combinations to try
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,  # Use all CPU cores
    verbose=1,
    random_state=42
)

# --- Perform randomized search on the training data ---
random_search.fit(X_train_relevant, y_train)

# --- Get the best model and its parameters ---
best_clf = random_search.best_estimator_
best_params = random_search.best_params_
print(f"Best hyperparameters: {best_params}")

# --- Evaluate on the validation set using the best model ---
y_val_proba = best_clf.predict_proba(X_val_relevant)[:, 1]  # Probability for the positive class
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set using the best model ---
y_test_proba = best_clf.predict_proba(X_test_relevant)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best hyperparameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 20}
Validation ROC AUC: 0.808, AUPRC: 0.435
Test ROC AUC: 0.765, AUPRC: 0.423


Without hyperparameter tuning:

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

# --- Definisci il modello Random Forest con parametri predefiniti ---
rf = RandomForestClassifier(random_state=42, n_jobs=-1)  # Usa tutti i core della CPU

# --- Addestra il modello sui dati di training ---
rf.fit(X_train_relevant, y_train)

# --- Valuta sul validation set ---
y_val_proba = rf.predict_proba(X_val_relevant)[:, 1]  # Probabilità per la classe positiva
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Valuta sul test set ---
y_test_proba = rf.predict_proba(X_test_relevant)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.793, AUPRC: 0.421
Test ROC AUC: 0.757, AUPRC: 0.398
